# CYTools tutorial notebook

<p align="center">
    <img src="https://cytools.liammcallistergroup.com/img/teacher.svg?sanitize=true" height="200"></img>
</p>

Let us look at the basics of the CYTools package. The starting objects for most computations are the ```Polytope``` and ```Cone``` classes. These can be imported as follows.

In [ ]:
from cytools import Polytope, Cone

Other important classes in this package are ```Triangulation``` and ```CalabiYau```. These should generally not be directly constructed by the user, and instead they are produced by designated functions.

Let us take a brief look at each of the classes.

## Polytopes

First, let's take a look at the ```Polytope``` class. A ```Polytope``` object can be created by specifying a list of points defining the convex hull. Note that CYTools only supports lattice polytopes so any floating point numbers will be truncated to integers.

In [ ]:
p = Polytope([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[-1,-1,-1,-1]])

We can print some information about the polytope as follows.

In [ ]:
p

The list of lattice points, boundary points, interior points, etc., can be computed by self-explanatory functions.

In [ ]:
p.points()

We can compute information relevant to Batyrev's construction of Calabi-Yau hypersurfaces when the polytope is reflexive and 4-dimensional. To avoid ambiguity, one must specify if the polytope should be viewed as living in the $M$ lattice or the $N$ lattice.

In [ ]:
p.h11(lattice="N"), p.h21(lattice="N")

## Using the Kreuzer-Skarke database

CYTools provides two useful functions to work with the Kreuzer-Skarke (KS) database. We can import them as follows.

In [ ]:
from cytools import read_polytopes, fetch_polytopes

The first function takes a file name as input and reads all polytopes specified in the format used in the KS database. The second file directly fetches the polytopes from the database. For example let's fetch 100 polytopes with $h^{2,1}$=7.

In [ ]:
g = fetch_polytopes(h21=7, lattice="N", limit=100)
g

As you can see above, these functions return generator objects that give one polytope at a time. To get the polytopes we can use

In [ ]:
p1 = next(g)
p2 = next(g)

Or to get the full list of polytopes we can use

In [ ]:
l = list(g)
len(l)

In this example generator had a limit of 100 polytopes, but since it had already generated two in the above it produced a list of only 98.

If you are not familiar with Python, it is worth noting that generators raise and exception once they reach the end. For this reason, if you are using the ```next``` function in your code it is usually necessary to wrap this with try-except statements as in the following example.

In [ ]:
g = fetch_polytopes(h21=2, lattice="N", limit=100)
for i in range(100):
    try:
      p = next(g)
      print(f"Fetched polytope {i}")
    except StopIteration:
      print(f"Iteration stopped at number {i}")
      break

Alternatively, one can conveniently use generators in the following way.

In [ ]:
g = fetch_polytopes(h21=2, lattice="N", limit=100)
for p in g:
    # Do something
    print(p)

## Triangulations

Let us now look at how we can triangulate the polytopes. We start with the following polytope

In [ ]:
p = Polytope([[1,0,0,0],[0,1,0,0],[0,0,1,0],[-1,1,1,0],[0,-1,-1,0],[0,0,0,1],[1,-2,1,1],[-2,2,0,-1],[1,0,-1,-1]])

We can obtain a triangulation simply by using

In [ ]:
t = p.triangulate()

And print information about the triangulation as follows.

In [ ]:
t

For 4-dimensional reflexive polytopes it defaults to finding an FRST of the points not interior to facets. Other options such as heights, whether to make it a star, the backend, etc., can be inputted as well.

In [ ]:
t = p.triangulate(heights=[0,3,7,1,9,1,1,1,3,2,2,2], make_star=True, backend="cgal")

Various properties of the triangulation can be accessed by self-explanatory functions.

In [ ]:
t.simplices()

Some of these require additional software that are included in the Docker image. For example, finding triangulations that differ by a bistellar flip requires TOPCOM.

In [ ]:
t.neighbor_triangulations()

If one wants to generate random triangulations, one for example can pick random heights around the Delaunay triangulation. This can be done with the ```random_triangulations_fast``` function.

In [ ]:
triangs = p.random_triangulations_fast(N=100)

Note: The above method to find triangulations is fast, but does not produce a fair sampling of triangulations. This can be done with the ```random_triangulations_fair``` function

## Calabi-Yaus

Let's now get to the class of most interest. A ```CalabiYau``` object can be obtained from a triangulation using.

In [ ]:
cy = t.get_cy()

Basic information can be printed as follows.

In [ ]:
cy

Various properties of the CY can be accessed by self-explanatory functions. For example, intersection numbers can be computed using

In [ ]:
intnums = cy.intersection_numbers()
intnums

## Cones

Lastly, let's briefly look at cones. These can be constructed by specifying a set of rays or normals to hyperplanes.

In [ ]:
c1 = Cone([[0,1],[1,1]])
c2 = Cone(hyperplanes=[[0,1],[1,1]])

Let us look at the Mori cone of the ambient toric variety above Calabi-Yau.

In [ ]:
mc = cy.ambient_mori_cone()

We can print some information about it as follows.

In [ ]:
mc

The ambient Kähler cone can be computed by taking the dual, or from the designated function.

In [ ]:
kc = cy.ambient_kahler_cone()

CYTools uses a lazy duality where no computation is done and instead the definition of the cone is dualized. This can be seen by printing the information.

In [ ]:
kc

However, we can still do the hard duality computation if desired.

In [ ]:
kc.rays()

And then the definition of the cone is updated.

In [ ]:
kc

We can compute the distance to the tip of the stretched Kähler cone as follows.

In [ ]:
kc.tip_of_stretched_cone(1)

This concludes the brief tutorial. We have some additional [advanced usage instructions](https://cytools.liammcallistergroup.com/docs/getting-started/advanced) for people who intend to perform large-scale computations with CYTools or who want to tinker with the Docker image. For a full list of available classes and functions please visit the [documentation tab](https://cytools.liammcallistergroup.com/docs/documentation/).